<table style="width:100%; background-color: #D9EDF7">
  <tr>
    <td style="border: 1px solid #CFCFCF">
      <b>Open Power System Data: Time series</b>
      <ul>
        <li><a href="http://data.open-power-system-data.org/datapackage_timeseries">Datapackage</a></li>
        <li><a href="main.ipynb">Main notebook</a></li>
        <li>Processing Notebook (this)</li>
      </ul>
    </td>
  </tr>
</table>

# Table of Contents
* [1. Introductory Notes](#1.-Introductory-Notes)
* [2. Settings](#2.-Settings)
	* [2.1 Import Python libraries](#2.1-Import-Python-libraries)
* [3. Download](#3.-Download)
	* [3.1 Select scope](#3.1-Select-scope)
	* [3.2 Automatic download (for all sources except Energinet.dk)](#3.2-Automatic-download-%28for-all-sources-except-Energinet.dk%29)
	* [3.3 Manual download (Energinet.dk)](#3.3-Manual-download-%28Energinet.dk%29)
* [4. Read](#4.-Read)
* [5. Processing](#5.-Processing)
	* [5.1 Missing Data Handling](#5.1-Missing-Data-Handling)
	* [5.2 Plausibility checks (not implemented)](#5.2-Plausibility-checks-%28not-implemented%29)
	* [5.3 Mark treated data (not implemented)](#5.3-Mark-treated-data-%28not-implemented%29)
	* [5.4 Rescale load data with data from www.ag-energiebilanzen.de (not implemented)](#5.4-Rescale-load-data-with-data-from-www.ag-energiebilanzen.de-%28not-implemented%29)
	* [5.5 Country specific calculations](#5.5-Country-specific-calculations)
		* [5.5.1 Calculate onshore wind generation for TenneT Germany](#5.5.1-Calculate-onshore-wind-generation-for-TenneT-Germany)
		* [5.5.2 Aggregate German data from individual TSOs](#5.5.2-Aggregate-German-data-from-individual-TSOs)
	* [5.6 Create hourly data from 15' data](#5.6-Create-hourly-data-from-15'-data)
	* [5.7 Insert a column with Central European (Summer-)time. (Not implemented)](#5.7-Insert-a-column-with-Central-European-%28Summer-%29time.-%28Not-implemented%29)
* [6. Create metadata](#6.-Create-metadata)
	* [6.1 General metadata](#6.1-General-metadata)
	* [6.2 Columns-specific metadata](#6.2-Columns-specific-metadata)
* [7. Write data to disk](#7.-Write-data-to-disk)
	* [7.1 Different shapes](#7.1-Different-shapes)
	* [7.2 Write to SQL-database](#7.2-Write-to-SQL-database)
	* [7.3 Write to Excel](#7.3-Write-to-Excel)
	* [7.4 Write to CSV](#7.4-Write-to-CSV)


# 1. Introductory Notes

This Notebook handles misssing and implausible data, performs calculations and aggragations and creates the output files. 

# 2. Settings

This section: load libraries, select scope. Takes <1s to run.

## 2.1 Import Python libraries

This section: load libraries and set up a log.

Note that this notebook makes use of the [pycountry](https://pypi.python.org/pypi/pycountry) library that is not part of Anaconda. Install it with with `pip install pycountry` from your command line.

In [1]:
from datetime import datetime, date, timedelta
import pandas as pd
import numpy as np
import logging
import pycountry
import json
import sqlite3
import yaml
import itertools

from timeseries_scripts.read import read
from timeseries_scripts.download import download 

# reload modules with execution of any code, to avoid having to restart 
# the kernel after editing timeseries_scripts
%load_ext autoreload
%autoreload 2 

logger = logging.getLogger('log')
logger.setLevel('INFO')

# 3. Download

This section: download data. Takes about XX hours to run.


Download sources are in `config/sources.yml`, which specifies, for each source, the variables (such as wind and solar generation) alongside all the parameters necessary to execute the downloads.

First, a data directory is created on your local computer. Then, download parameters for each data source are defined, including the URL. These parameters are then turned into a YAML-string. Finally, the download is executed one by one. If all data needs to be downloaded, this usually takes several hours.

Each file is saved under it's original filename. Note that the original file names are often not self-explanatory (called "data" or "January"). The files content is revealed by its place in the directory structure.

## 3.1 Select scope

This section: select the time range and the data sources for download. Default: all data sources implemented, full time range available.

In [5]:
# Optionally: Specify the beginning and end of the interval for which to attempt
# the download. If None, all available data will be downloaded
start_from_user = date(2013, 5, 17) #None  # i.e. date(2016, 1, 1)
end_from_user = date(2016, 8, 31) #None  # i.e. date(2016, 1, 31)

sources_yaml_path = 'input_parameters/sources.yml'
out_path = 'original_data3'

# Optionally, specify a subset to download/read, e.g. subset=['TenneT', '50Hertz']
subset = ['50Hertz', 'Amprion', 'TenneT', 'TransnetBW', 'Elia', 'ENTSO-E',
          'OPSD', 'Svenska Kraftnaet', 'Energinet.dk', 'CEPS', 'PSE']
subset = ['PSE']

## 3.2 Automatic download (for all sources except Energinet.dk)

In [ ]:
download(sources_yaml_path, 
         out_path, 
         start_from_user=start_from_user, 
         end_from_user=end_from_user, 
         subset=subset)

INFO:log:test 2013-05-23 17:00:10
INFO:log:Found local file: GEN_WIATR_20130517_20130523170026.csv
INFO:log:test 2013-05-24 17:00:10
INFO:log:Found local file: GEN_WIATR_20130518_20130524170027.csv
INFO:log:test 2013-05-25 17:00:10
INFO:log:Found local file: GEN_WIATR_20130519_20130525170020.csv
INFO:log:test 2013-05-26 17:00:10
INFO:log:Found local file: GEN_WIATR_20130520_20130526170021.csv
INFO:log:test 2013-05-27 17:00:10
INFO:log:Found local file: GEN_WIATR_20130521_20130527170017.csv
INFO:log:test 2013-05-28 17:00:10
INFO:log:Found local file: GEN_WIATR_20130522_20130528170020.csv
INFO:log:test 2013-05-29 17:00:10
INFO:log:Found local file: GEN_WIATR_20130523_20130529170019.csv
INFO:log:test 2013-05-30 17:00:10
INFO:log:Found local file: GEN_WIATR_20130524_20130530170023.csv
INFO:log:test 2013-05-31 17:00:10
INFO:log:Found local file: GEN_WIATR_20130525_20130531170020.csv
INFO:log:test 2013-06-01 17:00:10
INFO:log:Found local file: GEN_WIATR_20130526_20130601170023.csv
INFO:log:t

## 3.3 Manual download (Energinet.dk)

Energinet.dk data needs to be downloaded manually from http://www.energinet.dk/en/el/engrosmarked/udtraek-af-markedsdata/Sider/default.aspx
Check The following Boxes and then press the "Get extract"-button at the end of the page:

Period<br>
Get data from: 01-01-2000 To: Today<br>
all months

Data columns <br>
Elspot Price, Currency Code/MWh

- DK-West
- DK-East
- Norway
- Sweden (SE)
- Sweden (SE3)
- Sweden (SE4)
- DE European Power Exchange

Production and consumption, MWh/h

- DK-West: Wind power production
- DK-West: Solar cell production (estimated)
- DK-East: Wind power production
- DK-East: Solar cell production (estimated)
- DK: Wind power production (onshore)
- DK: Wind power production (offshore)

Data format:<br>
Currency code EUR<br>
Decimal format English number Format (period as decimal separator)<br>
Date format Other date format(YYYY-MM-DD)<br>
Recieve to Excel<br>

# 4. Read

This section: Read each downloaded file into a pandas-DataFrame and merge data from different sources if it has the same time resolution. Takes about 30 minutes to run.

These are the names of the rows at the top of the data used to store metadata internally. In the output data, this information will be moved to the datapackage.json File.

In [3]:
headers = ['variable', 'region', 'attribute', 'source', 'web']

In [18]:
%%time
out_path = 'original_data3'
data_sets = {'15min': pd.DataFrame(), '60min': pd.DataFrame()}

for key, data_to_add in read(sources_yaml_path, out_path, headers, subset=subset):
    if len(data_sets[key]) == 0:
        data_sets[key] = data_to_add
    else:
        data_sets[key] = data_sets[key].combine_first(data_to_add)

INFO:log:reading data:
	 Source:   PSE
	 Variable: wind
	 Filename: GEN_WIATR_20120526_20120601170031.csv
INFO:log:reading data:
	 Source:   PSE
	 Variable: wind
	 Filename: GEN_WIATR_20120527_20120602170027.csv
INFO:log:reading data:
	 Source:   PSE
	 Variable: wind
	 Filename: GEN_WIATR_20120528_20120603170023.csv
INFO:log:reading data:
	 Source:   PSE
	 Variable: wind
	 Filename: GEN_WIATR_20120529_20120604170023.csv
INFO:log:reading data:
	 Source:   PSE
	 Variable: wind
	 Filename: GEN_WIATR_20120530_20120605170015.csv
INFO:log:reading data:
	 Source:   PSE
	 Variable: wind
	 Filename: GEN_WIATR_20120531_20120606170017.csv
INFO:log:reading data:
	 Source:   PSE
	 Variable: wind
	 Filename: GEN_WIATR_20120601_20120607170019.csv
INFO:log:reading data:
	 Source:   PSE
	 Variable: wind
	 Filename: GEN_WIATR_20120602_20120608170028.csv
INFO:log:reading data:
	 Source:   PSE
	 Variable: wind
	 Filename: GEN_WIATR_20120603_20120609170028.csv
INFO:log:reading data:
	 Source:   PSE
	 Varia

AmbiguousTimeError: Cannot infer dst time from 2012-10-28 02:00:00 asthere are no repeated times

In [ ]:
#reindex with a synthetic index that is sure to be continous in order to expose gaps in the data
no_gaps = pd.DatetimeIndex(start=data_sets[param_dict['resolution']].index[0],
                           end=data_sets[param_dict['resolution']].index[-1],
                           freq=param_dict['resolution'])
data_sets[param_dict['resolution']] = data_sets[param_dict['resolution']].reindex(index=no_gaps)

Display the first five columns of the dataframes to get an impression of them.

In [22]:
data_sets['15min'].head()

variable,wind
region,PL
attribute,generation
source,PSE
web,http://www.pse.pl/index.php?modul=21&id_rap=24
timestamp,
2012-05-25 22:00:00,530.125
2012-05-25 23:00:00,467.775
2012-05-26 00:00:00,393.825
2012-05-26 01:00:00,319.625
2012-05-26 02:00:00,280.250


In [ ]:
data_sets['60min'].head()

Uncomment the following cell to save the DataFrames created by the read function to disk. This way you have the raw data to fall back to if something goes wrong in the ramainder of this notebook.

In [ ]:
#data_sets['15min'].to_pickle('data_sets_15.pickle')
#data_sets['60min'].to_pickle('data_sets_60.pickle')

Uncomment this to load the DataFrames saved above

In [ ]:
data_sets = {}
data_sets['15min'] = pd.read_pickle('data_sets_15.pickle')
data_sets['60min'] = pd.read_pickle('data_sets_60.pickle')

In [ ]:
data_sets['60min'].loc['2001-12-31 21:00':,'price']

# 5. Processing

This section: missing data handling, aggregation of sub-national to national data, aggregate 15'-data to 60'-resolution. Takes XX hours to run.

## 5.1 Missing Data Handling

Patch missing data. At this stage, only implemented for 15 minute resolution solar/wind in-feed data from German TSOs. Small gaps (up to 2 hours) are filled by linear interpolation. For the generation timeseries, larger gaps are guessed by up-/down scaling the data from other balancing areas to fit the expected magnitude of the missing data.

The locations of missing data are stored in the nan_table DataFrame.

In [ ]:
def find_NaN(frame, patch=False):
    '''
    Search for missing values in a DataFrame and optionally apply further 
    functions on each column.
    Returns the DataFrame with gaps patched and another DataFrame containing
    information about missing data.
    
    Parameters
    ----------    
    frame : pandas.core.frame.DataFrame
        DataFrame to inspect and possibly patch
    patch : boolean, optional
        If False, return unaltered DataFrame,
        if True, return patched DataFrame
    
    '''
    nan_table = pd.DataFrame()
    patched = pd.DataFrame()
    one_period = frame.index[1] - frame.index[0]
    for col_name, col in frame.iteritems():
        col = col.to_frame()  # kann man colname wieder an df drankleben? df sollte col heißen

        # tag all occurences of NaN in the data
        # (but not before first or after last actual entry)
        col['tag'] = (
            (col.index >= col.first_valid_index()) &
            (col.index <= col.last_valid_index()) &
            col.isnull().transpose().as_matrix()
        ).transpose()

        # make another DF to hold info about each region
        nan_regs = pd.DataFrame()

        # first row of consecutive region is a True preceded by a False in tags
        nan_regs['start_idx'] = col.index[
            col['tag'] & ~ 
            col['tag'].shift(1).fillna(False)]

        # last row of consecutive region is a False preceded by a True   
        nan_regs['till_idx'] = col.index[
            col['tag'] & ~ 
            col['tag'].shift(-1).fillna(False)] 
        
        if not col['tag'].any():
            logger.info('%s : nothing to patch in this column', col_name[0:3])
            col.drop('tag', axis=1, inplace=True)
            nan_idx = pd.MultiIndex.from_arrays([
                    [0, 0, 0, 0],
                    ['count', 'span', 'start_idx', 'till_idx']])
            nan_list = pd.DataFrame(index=nan_idx, columns=col.columns)
            
        else:
            # how long is each region
            nan_regs['span'] = (
                nan_regs['till_idx'] - nan_regs['start_idx'] + one_period)
            nan_regs['count'] = (nan_regs['span'] / one_period)
            # sort the nan_regs DataFtame to put longest missing region on top
            nan_regs = nan_regs.sort_values(
                'count', ascending=False).reset_index(drop=True)
            
            col.drop('tag', axis=1, inplace=True)
            nan_list = nan_regs.stack().to_frame()
            nan_list.columns = col.columns
            
            if patch:
                col = choose_fill_method(
                    col, col_name, nan_regs, frame, one_period)
                
        if len(patched) == 0:
            patched = col
        else:
            patched = patched.combine_first(col)

        if len(nan_table) == 0:
            nan_table = nan_list
        else:
            nan_table = nan_table.combine_first(nan_list)

    nan_table.columns.names = headers
    patched.columns.names = headers

    return patched, nan_table


def choose_fill_method(col, col_name, nan_regs, frame, one_period):
    '''
    Choose how a region of missing values should be treated. Returns
    the column with all nan_regs treated.
        
    Parameters
    ----------  
    col : pandas.core.frame.DataFrame
        A column from frame as a separate DataFrame 
    col_name : str
        DataFrame to inspect
    nan_regs : : pandas.core.frame.DataFrame
        DataFrame with each row representing a region of missing data in col
    frame : pandas.core.frame.DataFrame
        DataFrame to patch
    one_period : pandas.tslib.Timedelta
        Time resolution of frame and col (15/60 minutes)
        
    '''
    for i, nan_region in nan_regs.iterrows():
        j = 0
        # Interpolate missing value spans up to 2 hours
        if nan_region['span'] <= timedelta(hours=2):
            col = my_interpolate(i, j, nan_region, col, col_name, nan_regs, one_period)
        # Guess missing value spans longer than one hour based on other tsos
        # (Only for German wind and solar generation data)
        elif col_name[1][:2] == 'DE' and col_name[2] == 'generation':
            j += 1
            col = impute(nan_region, col, col_name, nan_regs, frame, one_period)
    
    return col


def my_interpolate(i, j, nan_region, col, col_name, nan_regs, one_period):
    '''
    Interpolate one missing value region in one column as described by 
    nan_region. Returns the column as a pandas.DataFrame.
    
    Parameters
    ----------
    i : int
        Counter for total number of regions of missing data
    j : int
        Counter for number regions of missing data not treated by by this
        function
    nan_region : pandas.core.series.Series
        Contains information on one region of missing data in col
    col : pandas.core.frame.DataFrame
        A column from frame as a separate DataFrame 
    col_name : str
        DataFrame to inspect
    nan_regs : : pandas.core.frame.DataFrame
        DataFrame with each row representing a region of missing data in col
    one_period : pandas.tslib.Timedelta
        Time resolution of frame and col (15/60 minutes)
    
    '''
    if i + 1 == len(nan_regs):
        logger.info('%s : \n\t '
                    'interpolated %s up-to-2-hour-spans of NaNs',
                    col_name[0:3], i + 1 - j)
        
    to_fill = slice(nan_region['start_idx'] - one_period,
                    nan_region['till_idx'] + one_period)
    col.iloc[:,0].loc[to_fill] = col.iloc[:,0].loc[to_fill].interpolate()
    
    return col


def impute(nan_region, col, col_name, nan_regs, frame, one_period):
    '''
    Impute missing value spans longer than one hour based on other TSOs.
        
    Parameters
    ----------
    nan_region : pandas.core.series.Series
        Contains information on one region of missing data in col
    col : pandas.DataFrame
        A column from frame as a separate DataFrame 
    col_name : str
        DataFrame to inspect
    nan_regs : : pandas.DataFrame
        DataFrame with each row representing a region of missing data in col
    frame : pandas.DataFrame
        DataFrame to patch
    one_period : pandas.tslib.Timedelta
        Time resolution of frame and col (15/60 minutes)
    
    '''
    #logger.info('guessed %s entries after %s', row['count'], row['start_idx'])
    day_before = pd.DatetimeIndex(
        freq='15min',
        start=nan_region['start_idx'] - timedelta(hours=24),
        end=nan_region['start_idx'] - one_period)

    to_fill = pd.DatetimeIndex(
        freq='15min',
        start=nan_region['start_idx'],
        end=nan_region['till_idx'])

    # other_tsos = [c[1] for c in compact.drop(col_name, axis=1).loc[:,(col_name[0],slice(None),col_name[2])].columns.tolist()]
    other_tsos = [
        tso for tso in ['DE-50Hertz', 'DE-Amprion', 'DE-TenneT', 'DE-TransnetBW']
        if tso != col_name[1]]

    # select columns with data for same technology (wind/solar) but from other TSOs
    similar = frame.loc[:,(col_name[0],other_tsos,col_name[2])]
    # calculate the sum using columns without NaNs the day 
    # before or during the period to be guessed
    similar = similar.dropna(
        axis=1,
        how='any',
        subset=day_before.append(to_fill)
    ).sum(axis=1)
    # calculate scaling factor for other TSO data
    factor =  similar.loc[day_before].sum(axis=0) / col.loc[day_before,:].sum(axis=0)

    guess = similar.loc[to_fill] / float(factor)
    col.iloc[:,0].loc[to_fill] = guess
    a = float(col.iloc[:,0].loc[nan_region['start_idx'] - one_period])
    b = float(col.iloc[:,0].loc[nan_region['start_idx']])
    if a == 0:
        deviation = '{} absolut'.format(a - b)
    else:
        deviation = '{:.2f} %'.format((a - b) / a * 100)
    logger.info(
        '%s : \n        '
        'guessed %s entries after %s \n        '
        'last non-missing: %s \n        '
        'first guessed: %s \n        '
        'deviation of first guess from last known value: %s',
        col_name[0:3], nan_region['count'], nan_region['start_idx'], a, b, deviation
    )                      
    return col

Patch the 15 minutes dataset and display the location of missing Data in the original data.

In [ ]:
# FRAUKE: Hier und bei den beiden nachfolgenden Befehlen kommt bei mir ein Fehler:
# IndexError: index 1 is out of bounds for axis 0 with size 0
# Ich hatte nur das Svenska Data set ausgewählt, liegt vermutlich daran
# Wäre gut hier trotzdem kein Fehler zu bekommen
patched60, nan_table60 = find_NaN(data_sets['60min'], patch=True)

Show the complete table of regions of missing values

In [ ]:
nan_table60

Execute this to see whether there is still missing data. This is the case for some of the forecast columns.

In [ ]:
patched2, nan_table2 = find_NaN(patched)
nan_table2

Execute this to see an example of where the data has been patched.

In [ ]:
# FRAUKE: Ist da immer ein Fehler? Könnte man da nicht den ersten gefundenen
# und gefüllten Fehler nehmen?
data_sets['15min'].loc['2015-10-24 23:00:00':'2015-10-25 03:00:00', 'wind-onshore']

In [ ]:
patched.loc['2015-10-24 23:00:00':'2015-10-25 03:00:00', 'wind']

Replace the untreated data set with the patched one.

In [ ]:
data_sets['60min'] = patched60

## 5.2 Plausibility checks (not implemented)

In [ ]:
# pv = compact.xs(('solar'), level=('variable'), axis=1, drop_level=False)
# pv.index = pd.MultiIndex.from_arrays([pv.index.date, pv.index.time], names=['date','time'])
# pv

# pv.groupby(level='time').max()

# pv.unstack().idxmax().to_frame().unstack().transpose()

## 5.3 Mark treated data (not implemented)

## 5.4 Rescale load data with data from www.ag-energiebilanzen.de (not implemented)

## 5.5 Country specific calculations

### 5.5.1 Calculate onshore wind generation for TenneT Germany

In [ ]:
new_col_header = ('wind-onshore', 'DE-TenneT', 'generation',
                  'own calculation', 'own calculation')

data_sets['15min'][new_col_header] = (
    data_sets['15min'][('wind-total', 'DE-TenneT', 'generation')] - 
    data_sets['15min'][('wind-offshore', 'DE-TenneT', 'generation')])

In [ ]:
# DELETE THIS 
# some more display options...sortlevel() solves the error where .loc[] requires
# a lexsortet index 
data_sets['15min'].sortlevel(axis=1).loc[:,('wind-offshore', slice(None), 'generation')]
data_sets['15min'].sortlevel(axis=1).loc[:,(slice(None), 'DE-50Hertz', 'generation')]
data_sets['15min'].drop('solar', axis=1, level=0).xs('DE-TenneT', axis=1, level=1)['2015':]

### 5.5.2 Aggregate German data from individual TSOs and calculate availabilities/profiles

The wind and solar in-feed data for the 4 German balancing areas is summed up and stored in in new columns, which are then used to calculate profiles, that is, the share of wind/solar capacity producing at a given time. The column headers are created in the fashion introduced in the read script.

In [ ]:
# FRAUKE: Funktioniert da auch wenn man die Deutschen Daten gar nicht alle 
# ausgewählt hat oder kommt dann ein Fehler? Falls vieles drauf basiert, dass
# die deutschen Daten mit dabei sind, könntest Du auch oben bei der Auswahl
# include_sources klar machen, dass die auf jeden Fall dabei sein müssen
# FRAUKE: Im Text vor dieser Zelle verstehe ich nicht was die profiles sind
# Einspeisung / Load für jeden Zeitschritt?
for tech in ['solar', 'wind-onshore', 'wind-offshore']:
    for attribute in ['generation']:  #, 'forecast']:
        sum_col = pd.Series()
        for tso in ['DE-50Hertz', 'DE-Amprion', 'DE-TenneT', 'DE-TransnetBW']:
            try:
                add_col = data_sets['15min'][tech, tso, attribute]
                if len(sum_col) == 0:
                    sum_col = add_col
                else:
                    sum_col = sum_col + add_col.values
            except KeyError: # Occurs i.e. for Amprion and TransnetBW, which
                # don't have offshore
                pass
                
        # Create a new MultiIndex
        tuples = [(tech, 'DE', attribute, 'own calculation', 'own calculation')]
        columns = pd.MultiIndex.from_tuples(tuples, names=headers)
        sum_col.columns = columns
        data_sets['15min'] = data_sets['15min'].combine_first(sum_col)
        
        # Calculate the profile column
        try:
            if attribute == 'generation':
                profile_col = sum_col.values / data_sets['15min'][tech, 'DE', 'capacity']
                tuples = [(tech, 'DE', 'profile', 'own calculation', 'own calculation')]
                columns = pd.MultiIndex.from_tuples(tuples, names=headers)
                profile_col.columns = columns
                data_sets['15min'] = data_sets['15min'].combine_first(profile_col)
        except KeyError:
            pass  # FIXME Jan: FIXME! JONATHAN: This step requires now capacities
        # seperately for on- and offshore from Flensburg

New columns for the aggregated data have been added to the 15 minutes dataset.

In [ ]:
# FRAUKE: bei 50Hertz gibt es noch immer NaN gleich am Anfang
data_sets['15min'].head()

## 5.6 Create hourly data from 15' data

The German renewables in-feed data comes in 15-minute intervals. We resample it to hourly intervals in order to match the load data from ENTSO-E.

In [ ]:
# FRAUKE: Hier kommt bei mir ein Fehler als ich es mit nur dem 50Hertz
# Datensatz probiert habe
resampled = data_sets['15min'].resample('H').mean()
try:
    data_sets['60min'] = data_sets['60min'].combine_first(resampled)
except KeyError:
    data_sets['60min'] = resampled

New columns for the resampled data have been added to the 60 minutes dataset.

In [ ]:
data_sets['60min'].head()

## 5.7 Insert a column with Central European (Summer-)time. (Not implemented)

In [ ]:
# Still causes some problems, not recommended
#for res_key, df in data_sets.items():
#    if not df.empty:
#        df.insert(0, 'cet-timestamp', df.index.tz_localize('UTC').tz_convert('Europe/Brussels'))

# 6. Create metadata

This section: create the metadata, both general and column-specific. All metadata we be stored as a JSON file. Takes <1s to run.

## 6.1 General metadata

First, we define the general metadata for the timeseries datapackage

In [ ]:
metadata_head = '''
name: opsd-timeseries

title: 'Time-series data: load, wind and solar, prices'

description: This data package contains different kinds of timeseries
    data relevant for power system modelling, namely electricity consumption 
    (load) for 36 European countries as well as wind and solar power generation
    and capacities and prices for a growing subset of countries. 
    The timeseries become available at different points in time depending on the
    sources. The full dataset is only available from 2012 onwards. The
    data has been downloaded from the sources, resampled and merged in
    a large CSV file with hourly resolution. Additionally, the data
    available at a higher resolution (Some renewables in-feed, 15
    minutes) is provided in a separate file. All data processing is
    conducted in python and pandas and has been documented in the
    Jupyter notebooks linked below.
opsd-jupyter-notebook-url: https://github.com/Open-Power-System-Data/datapackage_timeseries/blob/2016-07-14/main.ipynb

version: '2016-07-14'

opsd-changes-to-last-version: Included data from Energinet.DK, Elia and 
    Svenska Kraftnaet

keywords:
    - timeseries
    - electricity
    - in-feed
    - capacity
    - renewables
    - wind
    - solar
    - load
    - tso
    - europe
    - germany

geographical-scope: Europe

licenses: 
    - url: http://example.com/license/url/here
      version: 1.0
      name: License Name Here
      id: license-id-from-open

views: 
    - {}

maintainers:
    - web: http://example.com/
      name: Jonathan Muehlenpfordt
      email: muehlenpfordt@neon-energie.de

resources:
'''

source_template = '''
    - name: {source}
      web: {web}
'''

resource_template = '''
    - path: timeseries{res_key}.csv
      format: csv
      mediatype: text/csv
      alternative_formats:
          - path: timeseries{res_key}.csv
            stacking: Singleindex
            format: csv
          - path: timeseries{res_key}.xlsx
            stacking: Singleindex
            format: xlsx
          - path: timeseries{res_key}_multiindex.xlsx
            stacking: Multiindex
            format: xlsx
          - path: timeseries{res_key}_multiindex.csv
            stacking: Multiindex
            format: csv
          - path: timeseries{res_key}_stacked.csv
            stacking: Stacked
            format: csv
      schema:
          fields:
'''

indexfield = '''
            - name: timestamp
              description: Start of timeperiod in UTC
              type: datetime
              format: YYYY-MM-DDThh:mm:ssZ
'''

field_template = '''
            - name: {variable}_{region}_{attribute}
              description: {description}
              type: number (float)
              source:
                  name: {source}
                  web: {web}
              opsd-properties: 
                  Region: {region}
                  Variable: {variable}
                  Attribute: {attribute}
'''

descriptions_template = '''
load: Consumption in {geo} in MW
generation: Actual {tech} generation in {geo} in MW
actual: Actual {tech} generation in {geo} in MW
forecast: {tech} day-ahead generation forecast in {geo} in MW
capacity: {tech} capacity in {geo} in MW
profile: Share of {tech} capacity producing in {geo}
offshoreshare: {tech} actual offshore generation in {geo} in MW
EPEX: lalala
Elspot: dududu
'''

## 6.2 Columns-specific metadata

For each dataset/outputfile, the metadata has an entry in the "resources" list that describes the file/dataset. The main part of each entry is the "schema" dictionary, consisting of a list of "fields", meaning the columns in the dataset. The first field is the timestamp index of the dataset. For the other fields, we iterate over the columns of the MultiIndex index of the datasets to contruct the corresponding metadata.

In [ ]:
resource_list = '' # list of files included in the datapackage
source_list = '' # list of sources were data comes from
for res_key, df in data_sets.items():
    field_list = indexfield # list of of columns in a file, starting with the index field
    for col in df.columns: # create 
        h = {k: v for k, v in zip(headers, col)}
        if len(h['region']) > 2:
            geo = h['region'] + ' control area'
        elif h['region'] == 'NI':
            geo = 'Northern Ireland'
        elif h['region'] == 'CS':
            geo = 'Serbia and Montenegro'
        else:
            geo = pycountry.countries.get(alpha2=h['region']).name

        descriptions = yaml.load(
            descriptions_template.format(tech=h['variable'], geo=geo)
            )
        h['description'] = descriptions[h['attribute']]
        field_list = field_list + field_template.format(**h)
        source_list = source_list + source_template.format(**h)
    resource_list = resource_list + resource_template.format(res_key=res_key) + field_list

# Remove duplicates from sources_list. set() returns unique values from a 
# collection, butit cannot compare dicts. Since source_list is a list of of 
# dicts, this requires some juggling with data types
source_list = [dict(tupleized) 
               for tupleized in set(tuple(entry.items())
                                    for entry in yaml.load(source_list))] 

metadata = yaml.load(metadata_head)
metadata['sources'] = source_list
metadata['resources'] = yaml.load(resource_list)

Execute this to write the metadata to disk

In [ ]:
datapackage_json = json.dumps(metadata, indent=2, separators=(',', ': '))
with open('datapackage.json', 'w') as f:
    f.write(datapackage_json)

# 7. Write data to disk

This section: Save as [Data Package](http://data.okfn.org/doc/tabular-data-package) (data in CSV, metadata in JSON file). All files are saved in the directory of this notebook. Alternative file formats (SQL, XLSX) are also exported. Takes about 1 hour to run.

## 7.1 Different shapes

Data are provided in three different "shapes": 
* Singleindex (easy to read for humans, compatible with datapackage standard, small file size) 
* Multiindex (easy to read into GAMS, not compatible with datapackage standard, small file size) 
* Stacked (compatible with data package standard, large file size, many rows, too many for Excel) 

In [ ]:
data_sets_singleindex = {}
data_sets_multiindex = {}
data_sets_stacked = {}
for res_key, df in data_sets.items():
    if not df.empty:
        df_singleindex = df.copy()
        # Jan: List comprehensions are usually one-liner.
        # use first 3 levels of multiindex to create singleindex
        df_singleindex.columns = ['_'.join(col[0:3])
                                  for col
                                  in df.columns.values
                                 ]
        data_sets_singleindex[res_key] = df_singleindex

        data_sets_multiindex[res_key + '_multiindex'] = df

        stacked = df.copy()
        stacked.columns = stacked.columns.droplevel(['source', 'web'])
        stacked = stacked.transpose().stack(dropna=True).to_frame(name='data')
        data_sets_stacked[res_key + '_stacked'] = stacked

## 7.2 Write to SQL-database

This file format is required for the filtering function on the OPSD website. This takes about 30 seconds to complete.

In [ ]:
%%time 
for res_key, df in data_sets_singleindex.items():
    f = 'timeseries' + res_key
    df = df.copy()
    df.index = df.index.strftime('%Y-%m-%dT%H:%M:%SZ')
    df.to_sql(f, sqlite3.connect(f + '.sqlite'),
              if_exists='replace', index_label='timestamp')

## 7.3 Write to Excel

This takes about 15 Minutes to complete.

In [ ]:
%%time
# itertoools.chain() allows iterating over multiple dicts at once
for res_key, df in itertools.chain( 
        #data_sets_singleindex.items(),
        data_sets_multiindex.items()
    ):
    f = 'timeseries' + res_key
    df.to_excel(f + '.xlsx', float_format='%.2f', merge_cells=True) #merge_cells=False)"

## 7.4 Write to CSV

This takes about 10 minutes to complete.

In [ ]:
%%time
for res_key, df in itertools.chain(
        data_sets_singleindex.items(),
        data_sets_multiindex.items(),
        #data_sets_stacked.items()
    ):
    f = 'timeseries' + res_key
    df.to_csv(f + '.csv', float_format='%.2f', date_format='%Y-%m-%dT%H:%M:%SZ')

# References

